In [34]:
import pandas as pd
import os
import numpy as np
import scipy
import math
import random
import sklearn
import nltk
import warnings
import matplotlib.pyplot as plt
from nltk.corpus import stopwords
from scipy.sparse import csr_matrix
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from scipy.sparse.linalg import svds
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import MultiLabelBinarizer

In [35]:
path="D:\\CDS513\\Assignment 1\\Data"
    
os.chdir(path)

# Read csv data file
# Data without feature standardization
df = pd.read_csv('Data_final.csv')

In [36]:
df

,User-id,Game-title,Genre,personal-rating
0,5250,Alien Swarm,Shooter,3
1,975449,Alien Swarm,Shooter,5
2,1950243,Alien Swarm,Shooter,3
3,2259650,Alien Swarm,Shooter,2
4,2753525,Alien Swarm,Shooter,2
...,...,...,...,...
42534,287824954,Close Your Eyes,Puzzle,5
42535,293303005,All Is Dust,Role-playing (RPG),5
42536,300248088,Sigils of Elohim,Puzzle,5
42537,306839937,Secret Files Tunguska,Point-and-click,5


# Finding # of interaction

In [37]:
users_interactions_count_df = df.groupby(['User-id', 'Game-title']).size().groupby('User-id').size()
print('# users: %d' % len(users_interactions_count_df))

users_with_enough_interactions_df = users_interactions_count_df[users_interactions_count_df >= 5]\
                                    .reset_index()[['User-id']]


users_with_few_interactions_df = users_interactions_count_df[users_interactions_count_df < 3]\
                                    .reset_index()[['User-id']]

print('# users with at least 5 interactions: %d' % len(users_with_enough_interactions_df))
print('# users with less than 3 interactions: %d' % len(users_with_few_interactions_df))

# users: 8476
# users with at least 5 interactions: 1719
# users with less than 3 interactions: 5936


In [38]:
interactions_from_selected_users_df = df.merge(users_with_enough_interactions_df, 
               how = 'right',
               left_on = 'User-id',
               right_on = 'User-id')

interactions_from_few_selected_users_df = df.merge(users_with_few_interactions_df, 
               how = 'right',
               left_on = 'User-id',
               right_on = 'User-id')

print('# of interactions from all users: %d' % len(df))
print('# of interactions from users with at least 5 interactions: %d' % len(interactions_from_selected_users_df))
print('# of interactions from users with less than 3 interactions: %d' % len(interactions_from_few_selected_users_df))

# of interactions from all users: 42539
# of interactions from users with at least 5 interactions: 32809
# of interactions from users with less than 3 interactions: 6932


In [39]:
def smooth_user_preference(x):
    return math.log(1+x, 2)
    
interactions_full_df = interactions_from_selected_users_df \
                    .groupby(['User-id', 'Game-title'])['personal-rating'].sum() \
                    .apply(smooth_user_preference).reset_index()
print('# of unique user/item interactions: %d' % len(interactions_full_df))
interactions_full_df.head(10)

# of unique user/item interactions: 32809


,User-id,Game-title,personal-rating
0,76767,Age of Empires II HD Edition,1.584963
1,76767,Call of Duty Black Ops,2.584963
2,76767,Call of Duty Modern Warfare 2,2.584963
3,76767,Call of Duty Modern Warfare 3,2.000000
4,76767,Call of Duty World at War,2.584963
5,76767,Counter-Strike,2.584963
6,76767,Counter-Strike Global Offensive,1.000000
7,76767,Counter-Strike Source,1.000000
8,76767,Portal 2,2.321928
9,76767,Rise of Nations Extended Edition,2.000000


# Split Dataset for training and testing

In [41]:
interactions_train_df, interactions_test_df = train_test_split(interactions_from_selected_users_df,
                                   stratify=interactions_from_selected_users_df['User-id'],
                                   test_size=0.20,
                                   random_state=42)

print('# interactions on Train set: %d' % len(interactions_train_df))
print('# interactions on Test set: %d' % len(interactions_test_df))

# interactions on Train set: 26247
# interactions on Test set: 6562


In [42]:
#Indexing by personId to speed up the searches during evaluation
interactions_full_indexed_df = interactions_from_selected_users_df.set_index('User-id')
interactions_train_indexed_df = interactions_train_df.set_index('User-id')
interactions_test_indexed_df = interactions_test_df.set_index('User-id')

In [8]:
def get_items_interacted(UserID, interactions_df):
    # Get the user's data and merge in the game information.
    interacted_items = interactions_df.loc[UserID]['Game-title']
    return set(interacted_items if type(interacted_items) == pd.Series else [interacted_items])

# Top-N accuracy metrics

In [9]:
#Top-N accuracy metrics consts
EVAL_RANDOM_SAMPLE_NON_INTERACTED_ITEMS = 100

class ModelEvaluator:
    def get_not_interacted_items_sample(self, UserID, sample_size, seed=42):
        interacted_items = get_items_interacted(UserID, interactions_full_indexed_df)
        all_items = set(df['Game-title'])
        non_interacted_items = all_items - interacted_items

        random.seed(seed)
        non_interacted_items_sample = random.sample(non_interacted_items, sample_size)
        return set(non_interacted_items_sample)

    def _verify_hit_top_n(self, item_id, recommended_items, topn):        
            try:
                index = next(i for i, c in enumerate(recommended_items) if c == item_id)
            except:
                index = -1
            hit = int(index in range(0, topn))
            return hit, index

    def evaluate_model_for_user(self, model, UserID):
        #Getting the items in test set
        interacted_values_testset = interactions_test_indexed_df.loc[UserID]
        if type(interacted_values_testset['Game-title']) == pd.Series:
            user_interacted_items_testset = set(interacted_values_testset['Game-title'])
        else:
            user_interacted_items_testset = set(interacted_values_testset['Game-title'])
        interacted_items_count_testset = len(user_interacted_items_testset) 

        #Getting a ranked recommendation list from a model for a given user
        user_recs_df = model.recommend_items(UserID, 
                                               items_to_ignore=get_items_interacted(UserID, 
                                                                                    interactions_train_indexed_df), 
                                                                                    topn=10000000000)

        hits_at_5_count = 0
        hits_at_10_count = 0
        #For each item the user has interacted in test set
        for item_id in user_interacted_items_testset:
            #Getting a random sample (100) items the user has not interacted 
            #(to represent items that are assumed to be no relevant to the user)
            non_interacted_items_sample = self.get_not_interacted_items_sample(UserID, 
                                                                          sample_size=EVAL_RANDOM_SAMPLE_NON_INTERACTED_ITEMS)

            #Combining the current interacted item with the 100 random items
            items_to_filter_recs = non_interacted_items_sample.union(set([item_id]))

            #Filtering only recommendations that are either the interacted item or from a random sample of 100 non-interacted items
            valid_recs_df = user_recs_df[user_recs_df['Game-title'].isin(items_to_filter_recs)]                    
            valid_recs = valid_recs_df['Game-title'].values
            #Verifying if the current interacted item is among the Top-N recommended items
            hit_at_5, index_at_5 = self._verify_hit_top_n(item_id, valid_recs, 5)
            hits_at_5_count += hit_at_5
            hit_at_10, index_at_10 = self._verify_hit_top_n(item_id, valid_recs, 10)
            hits_at_10_count += hit_at_10

        #Recall is the rate of the interacted items that are ranked among the Top-N recommended items, 
        #when mixed with a set of non-relevant items
        recall_at_5 = hits_at_5_count / float(interacted_items_count_testset)
        recall_at_10 = hits_at_10_count / float(interacted_items_count_testset)

        user_metrics = {'hits@5_count':hits_at_5_count, 
                          'hits@10_count':hits_at_10_count, 
                          'interacted_count': interacted_items_count_testset,
                          'recall@5': recall_at_5,
                          'recall@10': recall_at_10}
        return user_metrics

    def evaluate_model(self, model):
        #print('Running evaluation for users')
        people_metrics = []
        for idx, UserID in enumerate(list(interactions_test_indexed_df.index.unique().values)):
            #if idx % 100 == 0 and idx > 0:
            #    print('%d users processed' % idx)
            user_metrics = self.evaluate_model_for_user(model, UserID)  
            user_metrics['UserID'] = UserID
            people_metrics.append(user_metrics)
        print('%d users processed' % idx)

        detailed_results_df = pd.DataFrame(people_metrics) \
                            .sort_values('interacted_count', ascending=False)
        
        global_recall_at_5 = detailed_results_df['hits@5_count'].sum() / float(detailed_results_df['interacted_count'].sum())
        global_recall_at_10 = detailed_results_df['hits@10_count'].sum() / float(detailed_results_df['interacted_count'].sum())
        
        global_metrics = {'modelName': model.get_model_name(),
                          'recall@5': global_recall_at_5,
                          'recall@10': global_recall_at_10}    
        return global_metrics, detailed_results_df
    
model_evaluator = ModelEvaluator()

# interactions on Train set: 26247
# interactions on Test set: 6562


# Content-based CF

In [25]:
# Ensure Genre are properly split into lists
df1 = df.copy()
df1['Genre'] = df1['Genre'].apply(lambda x: x.split(','))

# Use MultiLabelBinarizer to one-hot encode the Genre
mlb = MultiLabelBinarizer()
genre_matrix = mlb.fit_transform(df1['Genre'])

# Convert genre_matrix to a DataFrame for better readability (optional)
genre_matrix_df = pd.DataFrame(genre_matrix, columns=mlb.classes_, index=df1['Game-title'])

print("Genre Matrix:")
print(genre_matrix_df)

Genre Matrix:
                       Adventure  Arcade  Card & Board Game  Fighting  \
Game-title                                                              
Alien Swarm                    0       0                  0         0   
Alien Swarm                    0       0                  0         0   
Alien Swarm                    0       0                  0         0   
Alien Swarm                    0       0                  0         0   
Alien Swarm                    0       0                  0         0   
...                          ...     ...                ...       ...   
Close Your Eyes                0       0                  0         0   
All Is Dust                    0       0                  0         0   
Sigils of Elohim               0       0                  0         0   
Secret Files Tunguska          0       0                  0         0   
SimpleRockets                  0       0                  0         0   

                       Hack and slas

In [45]:
# Function to get item profile (genre vector) based on game title
def get_item_profile(genre):
    idx = df1[df1['Genre'] == genre].index[0]
    item_profile = genre_matrix[idx:idx+1]  # Ensure this returns a 2D array
    return scipy.sparse.csr_matrix(item_profile)  # Ensure it's a sparse matrix

# Function to get item profiles (genre vectors) for a list of game titles
def get_item_profiles(game_titles):
    item_profiles_list = [get_item_profile(x) for x in game_titles]
    item_profiles = scipy.sparse.vstack(item_profiles_list)
    return item_profiles

# Function to build user profile based on their interactions
def build_users_profile(person_id, interactions_indexed_df):
    interactions_person_df = interactions_indexed_df.loc[person_id]
    user_item_profiles = get_item_profiles(interactions_person_df['Game-title'])
    
    user_item_strengths = np.array(interactions_person_df['personal-rating']).reshape(-1, 1)
    
    # Weighted average of item profiles by the interactions strength
    user_item_strengths_weighted_avg = np.sum(user_item_profiles.multiply(user_item_strengths), axis=0) / np.sum(user_item_strengths)
    
    # Convert to numpy array and ensure 2D shape
    user_item_strengths_weighted_avg = np.asarray(user_item_strengths_weighted_avg).reshape(1, -1)
    
    # Normalize
    user_profile_norm = sklearn.preprocessing.normalize(user_item_strengths_weighted_avg)
    
    return user_profile_norm

# Function to build user profiles for all users
def build_users_profiles(): 
    interactions_indexed_df = interactions_train_df[interactions_train_df['Genre'] \
                                                   .isin(df1['Genre'])].set_index('User-id')
    user_profiles = {}
    for person_id in interactions_indexed_df.index.unique():
        user_profiles[person_id] = build_users_profile(person_id, interactions_indexed_df)
    return user_profiles

# Build user profiles
with warnings.catch_warnings():
    warnings.filterwarnings("ignore", category=DeprecationWarning)
    user_profiles = build_users_profiles()


In [52]:
class ContentBasedRecommender:
    
    MODEL_NAME = 'Content-Based'
    
    def __init__(self, items_df=None):
        self.items_df = items_df
        self.item_ids = list(items_df['Genre'])
        
    def get_model_name(self):
        return self.MODEL_NAME
        
    def _get_similar_items_to_user_profile(self, person_id, topn=1000):
        # Computes the cosine similarity between the user profile and all item profiles
        if person_id not in user_profiles:
            return []

        user_profile = user_profiles[person_id]
        cosine_similarities = cosine_similarity(user_profile, genre_matrix)
        
        # Gets the top similar items
        similar_indices = cosine_similarities.argsort().flatten()[-topn:]
        
        # Sort the similar items by similarity
        similar_items = sorted([(self.item_ids[i], cosine_similarities[0, i]) \
                                for i in similar_indices], key=lambda x: -x[1])
        return similar_items
        
    def recommend_items(self, user_id, items_to_ignore=[], topn=10, verbose=False):
        similar_items = self._get_similar_items_to_user_profile(user_id)
        
        # Ignores items the user has already interacted with
        similar_items_filtered = list(filter(lambda x: x[0] not in items_to_ignore, similar_items))
        
        recommendations_df = pd.DataFrame(similar_items_filtered, \
                                          columns=['Game-title', 'recStrength']).head(topn)

        if verbose:
            if self.items_df is None:
                raise Exception('"items_df" is required in verbose mode')

            # Ensure the 'Genre' column is stringified
            self.items_df['Genre'] = self.items_df['Genre'].apply(lambda x: ', '.join(x) if isinstance(x, list) else x)

            recommendations_df = recommendations_df.merge(self.items_df, how='left', 
                                                          left_on='Game-title', 
                                                          right_on='Game-title')[['recStrength', 
                                                                                  'Game-title', 
                                                                                  'Genre']]

        return recommendations_df.drop_duplicates()

In [53]:
# Assuming you have already prepared the 'df' DataFrame and built 'user_profiles'
content_based_recommender_model = ContentBasedRecommender(df1)

user_id = 975449  # Replace with an actual user ID
items_to_ignore = []  # List of items to ignore
recommendations = content_based_recommender_model.recommend_items(user_id, items_to_ignore, topn=20, verbose=True)

print("Recommendations:")
print(recommendations)

Recommendations:
   recStrength Game-title Genre
0     0.899277    Shooter   NaN


In [84]:
#Trains a model whose vectors size is 5000, composed by the main unigrams and bigrams found in the corpus, ignoring stopwords
vectorizer = TfidfVectorizer(analyzer='word',
                     ngram_range=(1, 2),
                     min_df=0.003,
                     max_df=0.5)

item_ids = df['Game-title'].tolist()
tfidf_matrix = vectorizer.fit_transform(df['Genre'])
tfidf_feature_names = vectorizer.get_feature_names_out()
tfidf_matrix

<42539x36 sparse matrix of type '<class 'numpy.float64'>'
	with 86705 stored elements in Compressed Sparse Row format>

In [85]:
def get_item_profile(item_id):
    idx = item_ids.index(item_id)
    item_profile = tfidf_matrix[idx:idx+1]
    return item_profile

def get_item_profiles(ids):
    item_profiles_list = [get_item_profile(x) for x in ids]
    item_profiles = scipy.sparse.vstack(item_profiles_list)
    return item_profiles

def build_users_profile(person_id, interactions_indexed_df):
    interactions_person_df = interactions_indexed_df.loc[person_id]
    user_item_profiles = get_item_profiles(interactions_person_df['Game-title'])
    
    user_item_strengths = np.array(interactions_person_df['personal-rating']).reshape(-1, 1)
    
    # Weighted average of item profiles by the interactions strength
    user_item_strengths_weighted_avg = np.sum(user_item_profiles.multiply(user_item_strengths), axis=0) / np.sum(user_item_strengths)
    
    # Convert to numpy array and ensure 2D shape
    user_item_strengths_weighted_avg = np.asarray(user_item_strengths_weighted_avg).reshape(1, -1)
    
    # Normalize
    user_profile_norm = sklearn.preprocessing.normalize(user_item_strengths_weighted_avg)
    
    return user_profile_norm

def build_users_profiles(): 
    interactions_indexed_df = interactions_train_df[interactions_train_df['Game-title'] \
                                                   .isin(df['Game-title'])].set_index('User-id')
    user_profiles = {}
    for person_id in interactions_indexed_df.index.unique():
        user_profiles[person_id] = build_users_profile(person_id, interactions_indexed_df)
    return user_profiles

# Example usage:
# Assuming interactions_train_df and articles_df are defined and contain the required data
with warnings.catch_warnings():
    warnings.filterwarnings("ignore", category=DeprecationWarning)
    user_profiles = build_users_profiles()

In [86]:
import warnings

with warnings.catch_warnings():
    warnings.filterwarnings("ignore", category=DeprecationWarning)
    
user_profiles = build_users_profiles()
len(user_profiles)

1719

In [87]:
myprofile = user_profiles[975449]
print(myprofile.shape)
pd.DataFrame(sorted(zip(tfidf_feature_names, 
                        user_profiles[975449].flatten().tolist()), key=lambda x: -x[1])[:20],
             columns=['token', 'relevance'])

(1, 36)


,token,relevance
0,shooter,0.898291
1,platform,0.290875
2,fighting,0.128327
3,playing,0.103302
4,playing rpg,0.103302
5,role,0.103302
6,role playing,0.103302
7,rpg,0.103302
8,adventure,0.085552
9,strategy,0.068353


In [88]:
class ContentBasedRecommender:
    
    MODEL_NAME = 'Content-Based'
    
    def __init__(self, items_df=None):
        self.item_ids = item_ids
        self.items_df = items_df
        
    def get_model_name(self):
        return self.MODEL_NAME
        
    def _get_similar_items_to_user_profile(self, person_id, topn=1000):
        #Computes the cosine similarity between the user profile and all item profiles
        cosine_similarities = cosine_similarity(user_profiles[person_id], tfidf_matrix)
        #Gets the top similar items
        similar_indices = cosine_similarities.argsort().flatten()[-topn:]
        #Sort the similar items by similarity
        similar_items = sorted([(item_ids[i], cosine_similarities[0,i]) for i in similar_indices], \
                               key=lambda x: -x[1])
        return similar_items
        
    def recommend_items(self, user_id, items_to_ignore=[], topn=10, verbose=False):
        similar_items = self._get_similar_items_to_user_profile(user_id)
        #Ignores items the user has already interacted
        similar_items_filtered = list(filter(lambda x: x[0] not in items_to_ignore, similar_items))
        
        recommendations_df = pd.DataFrame(similar_items_filtered, columns=['Game-title', 'recStrength']) \
                                    .head(topn)

        if verbose:
            if self.items_df is None:
                raise Exception('"items_df" is required in verbose mode')

            recommendations_df = recommendations_df.merge(self.items_df, how = 'left', 
                                                          left_on = 'Game-title', 
                                                          right_on = 'Game-title')[['recStrength', \
                                                                                   'Game-title', \
                                                                                   'Genre']]


        return recommendations_df.drop_duplicates()
    
content_based_recommender_model = ContentBasedRecommender(df)

In [89]:
import warnings

with warnings.catch_warnings():
    warnings.filterwarnings("ignore", category=DeprecationWarning)

print('Evaluating Content-Based Filtering model...')
cb_global_metrics, cb_detailed_results_df = model_evaluator.evaluate_model(content_based_recommender_model)
print('\nGlobal metrics:\n%s' % cb_global_metrics)
cb_detailed_results_df.head(10)

Evaluating Content-Based Filtering model...


C:\Users\Mackhem\AppData\Local\Temp\ipykernel_10592\902105647.py:10: DeprecationWarning: Sampling from a set deprecated
since Python 3.9 and will be removed in a subsequent version.
  non_interacted_items_sample = random.sample(non_interacted_items, sample_size)
C:\Users\Mackhem\AppData\Local\Temp\ipykernel_10592\902105647.py:10: DeprecationWarning: Sampling from a set deprecated
since Python 3.9 and will be removed in a subsequent version.
  non_interacted_items_sample = random.sample(non_interacted_items, sample_size)
C:\Users\Mackhem\AppData\Local\Temp\ipykernel_10592\902105647.py:10: DeprecationWarning: Sampling from a set deprecated
since Python 3.9 and will be removed in a subsequent version.
  non_interacted_items_sample = random.sample(non_interacted_items, sample_size)
C:\Users\Mackhem\AppData\Local\Temp\ipykernel_10592\902105647.py:10: DeprecationWarning: Sampling from a set deprecated
since Python 3.9 and will be removed in a subsequent version.
  non_interacted_items_sample

C:\Users\Mackhem\AppData\Local\Temp\ipykernel_10592\902105647.py:10: DeprecationWarning: Sampling from a set deprecated
since Python 3.9 and will be removed in a subsequent version.
  non_interacted_items_sample = random.sample(non_interacted_items, sample_size)
C:\Users\Mackhem\AppData\Local\Temp\ipykernel_10592\902105647.py:10: DeprecationWarning: Sampling from a set deprecated
since Python 3.9 and will be removed in a subsequent version.
  non_interacted_items_sample = random.sample(non_interacted_items, sample_size)
C:\Users\Mackhem\AppData\Local\Temp\ipykernel_10592\902105647.py:10: DeprecationWarning: Sampling from a set deprecated
since Python 3.9 and will be removed in a subsequent version.
  non_interacted_items_sample = random.sample(non_interacted_items, sample_size)
C:\Users\Mackhem\AppData\Local\Temp\ipykernel_10592\902105647.py:10: DeprecationWarning: Sampling from a set deprecated
since Python 3.9 and will be removed in a subsequent version.
  non_interacted_items_sample

C:\Users\Mackhem\AppData\Local\Temp\ipykernel_10592\902105647.py:10: DeprecationWarning: Sampling from a set deprecated
since Python 3.9 and will be removed in a subsequent version.
  non_interacted_items_sample = random.sample(non_interacted_items, sample_size)
C:\Users\Mackhem\AppData\Local\Temp\ipykernel_10592\902105647.py:10: DeprecationWarning: Sampling from a set deprecated
since Python 3.9 and will be removed in a subsequent version.
  non_interacted_items_sample = random.sample(non_interacted_items, sample_size)
C:\Users\Mackhem\AppData\Local\Temp\ipykernel_10592\902105647.py:10: DeprecationWarning: Sampling from a set deprecated
since Python 3.9 and will be removed in a subsequent version.
  non_interacted_items_sample = random.sample(non_interacted_items, sample_size)
C:\Users\Mackhem\AppData\Local\Temp\ipykernel_10592\902105647.py:10: DeprecationWarning: Sampling from a set deprecated
since Python 3.9 and will be removed in a subsequent version.
  non_interacted_items_sample

C:\Users\Mackhem\AppData\Local\Temp\ipykernel_10592\902105647.py:10: DeprecationWarning: Sampling from a set deprecated
since Python 3.9 and will be removed in a subsequent version.
  non_interacted_items_sample = random.sample(non_interacted_items, sample_size)
C:\Users\Mackhem\AppData\Local\Temp\ipykernel_10592\902105647.py:10: DeprecationWarning: Sampling from a set deprecated
since Python 3.9 and will be removed in a subsequent version.
  non_interacted_items_sample = random.sample(non_interacted_items, sample_size)
C:\Users\Mackhem\AppData\Local\Temp\ipykernel_10592\902105647.py:10: DeprecationWarning: Sampling from a set deprecated
since Python 3.9 and will be removed in a subsequent version.
  non_interacted_items_sample = random.sample(non_interacted_items, sample_size)
C:\Users\Mackhem\AppData\Local\Temp\ipykernel_10592\902105647.py:10: DeprecationWarning: Sampling from a set deprecated
since Python 3.9 and will be removed in a subsequent version.
  non_interacted_items_sample

...

C:\Users\Mackhem\AppData\Local\Temp\ipykernel_10592\902105647.py:10: DeprecationWarning: Sampling from a set deprecated
since Python 3.9 and will be removed in a subsequent version.
  non_interacted_items_sample = random.sample(non_interacted_items, sample_size)
C:\Users\Mackhem\AppData\Local\Temp\ipykernel_10592\902105647.py:10: DeprecationWarning: Sampling from a set deprecated
since Python 3.9 and will be removed in a subsequent version.
  non_interacted_items_sample = random.sample(non_interacted_items, sample_size)
C:\Users\Mackhem\AppData\Local\Temp\ipykernel_10592\902105647.py:10: DeprecationWarning: Sampling from a set deprecated
since Python 3.9 and will be removed in a subsequent version.
  non_interacted_items_sample = random.sample(non_interacted_items, sample_size)
C:\Users\Mackhem\AppData\Local\Temp\ipykernel_10592\902105647.py:10: DeprecationWarning: Sampling from a set deprecated
since Python 3.9 and will be removed in a subsequent version.
  non_interacted_items_sample

C:\Users\Mackhem\AppData\Local\Temp\ipykernel_10592\902105647.py:10: DeprecationWarning: Sampling from a set deprecated
since Python 3.9 and will be removed in a subsequent version.
  non_interacted_items_sample = random.sample(non_interacted_items, sample_size)
C:\Users\Mackhem\AppData\Local\Temp\ipykernel_10592\902105647.py:10: DeprecationWarning: Sampling from a set deprecated
since Python 3.9 and will be removed in a subsequent version.
  non_interacted_items_sample = random.sample(non_interacted_items, sample_size)
C:\Users\Mackhem\AppData\Local\Temp\ipykernel_10592\902105647.py:10: DeprecationWarning: Sampling from a set deprecated
since Python 3.9 and will be removed in a subsequent version.
  non_interacted_items_sample = random.sample(non_interacted_items, sample_size)
C:\Users\Mackhem\AppData\Local\Temp\ipykernel_10592\902105647.py:10: DeprecationWarning: Sampling from a set deprecated
since Python 3.9 and will be removed in a subsequent version.
  non_interacted_items_sample

1718 users processed

Global metrics:
{'modelName': 'Content-Based', 'recall@5': 0.026, 'recall@10': 0.0268}


C:\Users\Mackhem\AppData\Local\Temp\ipykernel_10592\902105647.py:10: DeprecationWarning: Sampling from a set deprecated
since Python 3.9 and will be removed in a subsequent version.
  non_interacted_items_sample = random.sample(non_interacted_items, sample_size)
C:\Users\Mackhem\AppData\Local\Temp\ipykernel_10592\902105647.py:10: DeprecationWarning: Sampling from a set deprecated
since Python 3.9 and will be removed in a subsequent version.
  non_interacted_items_sample = random.sample(non_interacted_items, sample_size)
C:\Users\Mackhem\AppData\Local\Temp\ipykernel_10592\902105647.py:10: DeprecationWarning: Sampling from a set deprecated
since Python 3.9 and will be removed in a subsequent version.
  non_interacted_items_sample = random.sample(non_interacted_items, sample_size)
C:\Users\Mackhem\AppData\Local\Temp\ipykernel_10592\902105647.py:10: DeprecationWarning: Sampling from a set deprecated
since Python 3.9 and will be removed in a subsequent version.
  non_interacted_items_sample

,hits@5_count,hits@10_count,interacted_count,recall@5,recall@10,UserID
51,0,0,62,0.000000,0.000000,62990992
145,0,0,49,0.000000,0.000000,138941587
31,2,2,36,0.055556,0.055556,48798067
302,1,1,36,0.027778,0.027778,11403772
132,1,1,35,0.028571,0.028571,49893565
30,2,2,35,0.057143,0.057143,24469287
440,0,0,32,0.000000,0.000000,116876958
304,0,0,30,0.000000,0.000000,47457723
63,3,9,29,0.103448,0.310345,11373749
56,0,0,28,0.000000,0.000000,10599862


In [91]:
def inspect_interactions(person_id, test_set=True):
    if test_set:
        interactions_df = interactions_test_indexed_df
    else:
        interactions_df = interactions_train_indexed_df
    return interactions_df.loc[person_id]

In [92]:
inspect_interactions(975449, test_set=False).head(20)

,Game-title,Genre,personal-rating
User-id,,,
975449,Thief Deadly Shadows,Shooter,4
975449,Awesomenauts,Platform,2
975449,Borderlands,Shooter,2
975449,Crysis Warhead,Shooter,3
975449,Torchlight,Role-playing (RPG),2
975449,VVVVVV,Platform,5
975449,Dead Island,Shooter,2
975449,Rock of Ages,Racing,5
975449,A.R.E.S.,Shooter,3


In [93]:
content_based_recommender_model.recommend_items(975449, topn=20, verbose=True)

,recStrength,Game-title,Genre
0,0.898291,Robocraft,Shooter
